<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/ColabQwenServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#script works on T4 only
!pip install flask-ngrok
!pip install flask
#apparently not needed

In [ ]:
!pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

In [2]:
#!pip install pyngrok==4.1.1
!pip install pyngrok
!pip install qwen-vl-utils[decord]
!pip install autoawq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.8-py3-none-any.whl size=108744 sha256=b05f975df2aa256e99bd6d07b48bdaa36fe013e5f2a3dd5ed236c485bcdb8007
  Stored in directory: /root/.cache/pip/wheels/fd/03/fe/99c1c678bfe8aca712186466969ed8

In [3]:
import requests
from io import BytesIO
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct-AWQ", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct-AWQ")




config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [11]:
def process_message_single(prompt, question, path_to_video):
  messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "file:///content/" + path_to_video, #sample_data/5fps_10s_quarter_video_example.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": question }, #"Is there person visible. Answer with 1 word: yes or no."},
        ],
    }
  ]
  print(messages)
  # Preparation for inference
  text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs = process_vision_info(messages)
  inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
  )
  inputs = inputs.to("cuda")

  # Inference
  generated_ids = model.generate(**inputs, max_new_tokens=128)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  print(output_text)
  return output_text



In [13]:


from pyngrok import ngrok
from flask import Flask, request, Response
import os

# token got to https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "2s2ir8cFjvMjxgrJcE3H2rlYehD_3dqAtUeXnvf7nax5mTV5J"

port = 5000
ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)

is_busy = False

@app.route("/")
def root():
  return "Hello world colab pyngrok!"

@app.route("/req1", methods=['POST'])
def req1():
    global is_busy
    if 'video' not in request.files or 'text' not in request.form:
        return "Missing video or text data", 400

    video_file = request.files['video']
    text_data = request.form['text']
    if is_busy:
      print("Server is busy, skipping 10s chunk processing")
      return "Server is busy", 503
    is_busy = True

    # Print the received data
    #print(f"Received text data: {text_data}")
    #print(f"Received video file: {video_file.filename}")

    # You can process the video file here, e.g., save it to disk
    videoPath = "sample_data/"+video_file.filename
    video_file.save(videoPath)
    response = process_message_single("", text_data, videoPath)
    # Display the video in the notebook (not possible with ngrok)
    # display(HTML(f'<video controls><source src="data:video/mp4;base64,{base64.b64encode(video_file.read()).decode()}" type="video/mp4"></video>'))

    os.remove(videoPath)
    # Send the text data back as response
    print( str(response))
    is_busy = False
    return response




if __name__ == "__main__":
  try:
    app.logger.info('Hello from Flask logger!')
    public_url = ngrok.connect(port).public_url
    print(public_url)
    app.logger.info(public_url)
    app.run(port=port)
  finally:
    ngrok.disconnect(public_url=public_url)



https://2725-35-230-15-249.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jan/2025 20:21:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2025 20:21:22] "GET /favicon.ico HTTP/1.1" 404 -


[{'role': 'user', 'content': [{'type': 'video', 'video': 'file:///content/sample_data/IntermediateOutput_638732641782812676.mp4', 'max_pixels': 151200, 'fps': 1.0}, {'type': 'text', 'text': 'Does cellphone appear in the film? Response with yes or no.'}]}]


INFO:werkzeug:127.0.0.1 - - [23/Jan/2025 20:23:11] "POST /req1 HTTP/1.1" 200 -


['no']
['no']
[{'role': 'user', 'content': [{'type': 'video', 'video': 'file:///content/sample_data/IntermediateOutput_638732642529978703.mp4', 'max_pixels': 151200, 'fps': 1.0}, {'type': 'text', 'text': 'Does cellphone appear in the film? Response with yes or no.'}]}]


...

INFO:werkzeug:127.0.0.1 - - [23/Jan/2025 20:29:53] "POST /req1 HTTP/1.1" 200 -


['No']
['No']


ERROR:__main__:Exception on /req1 [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-13-00f85cb698b4>", line 39, in req1
    response = process_message_single("", text_data, videoPath)
         

[{'role': 'user', 'content': [{'type': 'video', 'video': 'file:///content/sample_data/IntermediateOutput_638732645911077333.mp4', 'max_pixels': 151200, 'fps': 1.0}, {'type': 'text', 'text': 'Does cellphone appear in the film? Response with yes or no.'}]}]


RemoteDisconnected: Remote end closed connection without response